# CS4487 Project: Image classification

## Code Reproduction
* ### Environment Requirments
  * #### Python Version >= 3.7
  * #### Pytorch Version =1.3
  * #### Nvdia GPU (for speed up training and testing)
* ### Run Codes
  * #### Run all the code from top to bottom
  * #### The first code cell helps install a progress bar library

## Regarding course project
* External training data is not allowed. Learning can only be performed on the 50,000 training images in **CIFAR10**
* The [**CIFAR10** dataset](https://www.cs.toronto.edu/~kriz/cifar.html) was comprised of 60,000 32 × 32 color photographs of objects from 10 classes, with 6,000 images per class. There are 50,000 training images and 10,000 test images.
* Submit the results at [Kaggle CS4487 Course Project](https://www.kaggle.com/c/cs4487cp/)
* The second test set containing roughly 2,000 images (refer to test_data/y_test.npy)

## Evaluation Criteria 

* ### Classification Accuaracy:
  ### $$ Acc(f, D) = \frac{1}{m}\sum_{i=1}^{m}\mathbb{I}[y^i = f(x^i)]$$

* ### File Format:
Upload a file in CSV format, such as:

|         **Index**         |         **Category**        |
|---------------------------|-----------------------------|
| 0                         | 8                           |
| 1                         | 4                           |
| 2                         | 6                           |
| 3                         | 1                           |
| ......                    | ......                      |
| 11999                     | 2                           |

PyTorch Tutorial: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

---
## Table of content (for quick jump)
* ### [Import Library](#Import-Library)
* ### [Data Preparation](#Data-Preparation)
* ### [Common Functions](#Common-Functions)
  * #### [Training Function](#Training-function)
  * #### [Testing Function](#Testing-function)
  * #### [Test for each category](#Test-for-each-category)
  * #### [Save Model](#Save-Model)
  * #### [Print Time](#Print-Time)
  * #### [Progress Bar](#Progress-Bar)
* ### [DenseNet (DN)](#DenseNet)
  * #### [Training](#Train-DenaseNet)
  * #### [Testing](#Test-DenseNet)
* ### [Run the model](#Run-the-model-in-project-test-data)
---
---

## Import Library

In [ ]:
%pip install progressbar2

In [4]:
import numpy as np
from numpy import *
import cv2
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import matplotlib
import time
from _datetime import datetime
import pytz

import torch
import torchvision
from torch.utils import data
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torch.optim as optim

from progressbar import *
from sys import platform

import matplotlib.pyplot as plt

In [5]:
gd_root = "."  # For loacal run

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

basic_path = "basic_cifar_net.pth"
PATH_DPN = "DPN_cifar.pth"
PATH_Dense = "DenseNet.pth"

cuda


---
---
## Data Preparation

### Read CIFAR10 data
* The archive contains the files data_batch_1, data_batch_2, ..., data_batch_5, as well as test_batch. Each of these files is a Python "pickled" object produced with cPickle.
* Loaded in this way, each of the batch files contains a dictionary with the following elements:
  * **data**: 
    * a 10,000 x 3,072 numpy array of uint8s. Each row of the array stores a 32 x 32 colour image. 
    * The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. 
    * The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.
  * **labels**: 
    * a list of 10,000 numbers in the range 0 - 9. 
    * The number at index i indicates the label of the ith image in the array data.
* The dataset contains another file, called batches.meta. It too contains a Python dictionary object. It has the following entries:
  * **label_names** -- a 10-element list which gives meaningful names to the numeric labels in the labels array described above. 
  * For example, label_names[0] == "airplane", label_names[1] == "automobile", etc.

### Loading and normalizing CIFAR10 to dataloader

In [7]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [8]:
trainset = torchvision.datasets.CIFAR10(root=f'{gd_root}/data', train=True, download=False, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, 
                                           batch_size=64, 
                                           shuffle=True, 
                                           num_workers=8, 
                                           pin_memory=True
                                          )

testset = torchvision.datasets.CIFAR10(root=f'{gd_root}/data', train=False, download=False, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, 
                                          batch_size=32,
                                          shuffle=False, 
                                          num_workers=8,
                                          pin_memory=True
                                         )

In [30]:
train_size = len(trainset)
train_size
test_size = len(testset)
test_size
train_batch_len = len(train_loader)
train_batch_len
test_batch_len = len(test_loader)
test_batch_len

50000

10000

782

313

In [10]:
category_dict = {
    'airplane': 0, 
    'automobile': 1, 
    'bird': 2, 
    'cat': 3, 
    'deer': 4, 
    'dog': 5, 
    'frog': 6, 
    'horse': 7, 
    'ship': 8, 
    'truck': 9
}

---
---
## Common Functions

### Mixup 
From [Facebook AI Research](https://github.com/facebookresearch/mixup-cifar10/blob/master/README.md)

In [11]:
def mixup_data(x, y, alpha=1.0):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if device == 'cuda':
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

### Training function

In [52]:
train_correct_rate = []
train_loss = []

In [56]:
def training(epoch_num: int, model_name: str, net):    
    
    final_loss = 0.0
    
    total = train_size
    
    net.train()
    
    scheduler.step()
    
    for epoch in range(epoch_num):  # loop over the dataset multiple times
        
        correct = 0
        run_loss  = 0.0
        
        train_bar = progress_bar(train_batch_len)
        
        print_time_and_msg(f"Training for epoch {epoch + 1}")
        
        for batch_idx, (inputs, targets) in enumerate(train_loader, 0):
                        
            if device == 'cuda':
                inputs, targets = inputs.cuda(), targets.cuda()
            
            # Mixup
            inputs, targets_a, targets_b, lam = mixup_data(inputs, targets, 1.0)
            
            outputs = net(inputs)
            
            # forward + backward + optimize
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            run_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            
            final_loss = round(run_loss / train_batch_len, 3)
            
            train_bar.update(batch_idx)
        
        correct_rate = correct / total
        if epoch == 0:
            save_model(model_name, net)
        elif correct_rate > max(train_correct_rate):
            save_model(model_name, net)
        elif (epoch + 1) % 100 == 0:
            save_model(model_name, net)
        else:
            pass
        
        print_time_and_msg(f"Trained Epoch {epoch + 1}. loss: {final_loss}. Correct/Total: {correct}/{total}, Correct rate: {correct_rate}")
        train_correct_rate.append(correct_rate)
        train_loss.append(final_loss)
        print()
    
    print_time_and_msg(f'Finished training total {train_size} data for {epoch_num} epoches,\nthe final loss is {final_loss}')


### Testing function

In [16]:
def testing(net):
    test_bar = progress_bar(test_batch_len)
    correct = 0
    total = 0
    
    with torch.no_grad():
        for ti, (inputs, targets) in enumerate(test_loader):
            
            if device == 'cuda':
                inputs, targets = inputs.cuda(), targets.cuda()
                
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
            test_bar.update(ti)
    print()
    print(f'Accuracy of the network on the {test_size} test images: {100 * correct / total}%')



### Test for each category

In [17]:
def test_for_each(net):
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    
    test_bar = progress_bar(test_batch_len)
    
    with torch.no_grad():
        for tei, (inputs, targets) in enumerate(test_loader):
            
            if device == 'cuda':
                inputs, targets = inputs.cuda(), targets.cuda()
                
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == targets).squeeze()
            for i in range(4):
                label = targets[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
            
            test_bar.update(tei)
    
    print()
    for i in range(10):
        print(f'Accuracy of {list(category_dict)[i]} : {round(100 * class_correct[i] / class_total[i], 2)}%')

### Save Model

In [18]:
def save_model(model_name: str, net):
    PATH_basic = f'{gd_root}/models/{model_name}'
    torch.save(net.state_dict(), PATH_basic)
    print_time_and_msg(f"Saved model")

### Print Time

In [19]:
def print_time_and_msg(msg: str) -> str:
    now_time = time.time()
    readable_time = datetime.fromtimestamp(now_time, pytz.timezone('Asia/Shanghai')).strftime(f'[%H:%M:%S] - {msg}')
    print(readable_time)

### Progress Bar

In [20]:
def progress_bar(max_val: int):
    bar = ProgressBar(
        widgets=[
            'Progress: ',
            Percentage(), ' ', 
            Bar('#'), ' ', 
            Timer(), ' ', 
            ETA(), ' '
        ], 
        max_value=max_val
    )
    return bar


### Plot

In [21]:
def plot_train_info():
    plt.figure(figsize=(20,10))
    x = [i for i in range(len(train_correct_rate))]
    y1 = train_correct_rate
    y2 = train_loss
    
    plt.plot(x, y1, label="train correct rate")
    plt.plot(x, y2, label="train loss")
    
    plt.legend(loc=0, numpoints=1)
    
    plt.xlabel("Epoch")
    
# plot_train_info()

### Unpickle CIFAR data

In [22]:
def unpickle_cifar10(file: str) -> dict:
    full_name = f"{gd_root}/data/cifar-10-batches-py/{file}"
    
    with open(full_name, 'rb') as fi:
        cifar10_dict = pickle.load(fi, encoding='bytes')
    return cifar10_dict

cifar_meta = unpickle_cifar10("batches.meta")
cifar_meta

{b'num_cases_per_batch': 10000,
 b'label_names': [b'airplane',
  b'automobile',
  b'bird',
  b'cat',
  b'deer',
  b'dog',
  b'frog',
  b'horse',
  b'ship',
  b'truck'],
 b'num_vis': 3072}

---
---
## DenseNet

In [23]:
class Bottleneck(nn.Module):
    def __init__(self, in_planes, growth_rate):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, 4*growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4*growth_rate)
        self.conv2 = nn.Conv2d(4*growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat([out,x], 1)
        return out


class Transition(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_planes)
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=1, bias=False)

    def forward(self, x):
        out = self.conv(F.relu(self.bn(x)))
        out = F.avg_pool2d(out, 2)
        return out


class DenseNet(nn.Module):
    def __init__(self, block, nblocks, growth_rate=12, reduction=0.5, num_classes=10):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate

        num_planes = 2*growth_rate
        self.conv1 = nn.Conv2d(3, num_planes, kernel_size=3, padding=1, bias=False)

        self.dense1 = self._make_dense_layers(block, num_planes, nblocks[0])
        num_planes += nblocks[0]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans1 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense2 = self._make_dense_layers(block, num_planes, nblocks[1])
        num_planes += nblocks[1]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans2 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense3 = self._make_dense_layers(block, num_planes, nblocks[2])
        num_planes += nblocks[2]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans3 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense4 = self._make_dense_layers(block, num_planes, nblocks[3])
        num_planes += nblocks[3]*growth_rate

        self.bn = nn.BatchNorm2d(num_planes)
        self.linear = nn.Linear(num_planes, num_classes)

    def _make_dense_layers(self, block, in_planes, nblock):
        layers = []
        for i in range(nblock):
            layers.append(block(in_planes, self.growth_rate))
            in_planes += self.growth_rate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.trans3(self.dense3(out))
        out = self.dense4(out)
        out = F.avg_pool2d(F.relu(self.bn(out)), 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def DenseNet121():
    return DenseNet(Bottleneck, [6,12,24,16], growth_rate=32)

### Train DenaseNet

#### Go to [training function](#Training-function)

In [ ]:
densenet_net = DenseNet121()
densenet_net.to(device)
if device == 'cuda':
    densenet_net = torch.nn.DataParallel(densenet_net)
    cudnn.benchmark = True

In [69]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(densenet_net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

In [36]:
model_PATH = f"{gd_root}/models/{PATH_Dense}"
densenet_net.load_state_dict(torch.load(model_PATH))
print_time_and_msg(f"Loaded model: {PATH_Dense}")

<All keys matched successfully>

[18:28:38] - Loaded model: DenseNet.pth


In [28]:
epoches = 100

In [55]:
print_time_and_msg(f"Start training Dense Net for {epoches} epoches\n")
training(epoch_num=epoches, model_name=PATH_Dense, net=densenet_net)

[20:33:47] - Start training Dense Net for 100 epoches

[20:33:47] - Training for epoch 1


Progress:  99% |####################### | Elapsed Time: 0:01:06 ETA:   0:00:00 

[20:34:54] - Saved model
[20:34:54] - Trained Epoch 1. loss: 0.821. Correct/Total: 27201/50000, Correct rate: 0.54402

[20:34:54] - Training for epoch 2


Progress:  99% |####################### | Elapsed Time: 0:01:06 ETA:   0:00:00 

[20:36:01] - Saved model
[20:36:01] - Trained Epoch 2. loss: 0.817. Correct/Total: 27364/50000, Correct rate: 0.54728

[20:36:01] - Training for epoch 3


...

[20:53:36] - Trained Epoch 18. loss: 0.75. Correct/Total: 27513/50000, Correct rate: 0.55026

[20:53:36] - Training for epoch 19


Progress:  99% |####################### | Elapsed Time: 0:01:05 ETA:   0:00:00 

[20:54:43] - Trained Epoch 19. loss: 0.756. Correct/Total: 27117/50000, Correct rate: 0.54234

[20:54:43] - Training for epoch 20


Progress:  30% |#######                 | Elapsed Time: 0:00:19 ETA:   0:00:45 

KeyboardInterrupt: 

In [ ]:
plot_train_info()

### Test DenseNet

In [ ]:
model_PATH = f"{gd_root}/models/{PATH_Dense}"
test_densenet_net = DenseNet121()
test_densenet_net.to(device)
if device == 'cuda':
    test_densenet_net = torch.nn.DataParallel(test_densenet_net)
    cudnn.benchmark = True
test_densenet_net.load_state_dict(torch.load(model_PATH))

In [58]:
testing(test_densenet_net)

Progress:  82% |###################     | Elapsed Time: 0:00:08 ETA:   0:00:01 


Accuracy of the network on the 10000 test images: 94.94%


In [59]:
test_for_each(test_densenet_net)

Progress:  82% |###################     | Elapsed Time: 0:00:09 ETA:   0:00:02 


Accuracy of airplane : 95.58%
Accuracy of automobile : 98.26%
Accuracy of bird : 90.79%
Accuracy of cat : 89.15%
Accuracy of deer : 93.18%
Accuracy of dog : 93.81%
Accuracy of frog : 97.52%
Accuracy of horse : 98.44%
Accuracy of ship : 97.35%
Accuracy of truck : 97.06%


---
---
## Run the model in project test data 

### Load test data

In [61]:
project_testset = np.load(f"{gd_root}/data/test_data/y_test.npy").astype(np.uint8)
project_testset = np.moveaxis(project_testset, -1, 2)
project_testset = np.moveaxis(project_testset, 2, 1)
project_testset.shape
type(project_testset)

(12000, 3, 32, 32)

numpy.ndarray

### Convert to DataLoader ([Convertion Reference](https://stackoverflow.com/questions/44429199/how-to-load-a-list-of-numpy-arrays-to-pytorch-dataset-loader))

In [62]:
from torch.utils import data
tensor_testset = torch.Tensor(project_testset)
the_dataset = data.TensorDataset(tensor_testset)
project_test_loader = data.DataLoader(the_dataset, 
                                      batch_size=32,
                                      num_workers=8,
                                      pin_memory=True
                                     )
test_batch_len = len(project_test_loader)

### Run the test

In [63]:
predict_result = []

In [64]:
def run_project_test(net, model_path):
    net.load_state_dict(torch.load(model_path))
    
    test_pbar = progress_bar(test_batch_len)
    
    with torch.no_grad():
        for index, imgs in enumerate(project_test_loader):
            
            imgs = torch.stack([torch.Tensor(i) for i in imgs]).squeeze()
            
            if device == 'cuda':
                imgs.cuda()
                
            outputs = net(imgs)
            _, predicted = torch.max(outputs.data, 1)
            
            for p in predicted:
                predict_result.append(p.item())
            
            test_pbar.update(index)
      
    print_time_and_msg(f"Testd {len(predict_result)} images")

In [ ]:
model_PATH = f"{gd_root}/models/{PATH_Dense}"  # PATH_DPN
# p_test_net = DPN92()
p_test_net = DenseNet121()
if device == 'cuda':
    p_test_net.to(device)
    p_test_net = torch.nn.DataParallel(p_test_net)
    cudnn.benchmark = True
run_project_test(p_test_net, model_PATH)

### Save to csv file

In [66]:
def save_to_csv(result: list):
    result_df = pd.DataFrame(
        list(predict_result),
        columns=["Category"]
    )
    result_df.index.name = "Index"
    print(result_df.shape)
    result_df.to_csv(f"{gd_root}/predict_result.csv")

In [67]:
save_to_csv(predict_result)

(12000, 1)
